In [1]:
import ee
#ee.Authenticate()
ee.Initialize()

RefreshError: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})

In [2]:
#Import required packages
import geemap
import pandas as pd
import numpy as np
import time
import glob

In [3]:
airsheds = glob.glob("gridextents_shponly/*.shp")
airsheds=['gridextents_shponly\\grids_delhi.shp']

In [4]:
#global tifs
#tifs = glob.glob("bishek/NO2_tifs/*.tif")

In [5]:
# Chennai airshed box is already uploaded as a feature on GEE.
#chennai_box = ee.FeatureCollection("projects/ee-saikrishnadammalapati/assets/chennai-box")
#aoi = chennai_box.geometry()

def get_aoi(airshed_shp):
    airshed_box = geemap.shp_to_ee(airshed_shp)
    aoi = airshed_box.geometry()
    return airshed_box, aoi

# The following is a feature at all India level.
#admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
#india = admin2.filter(ee.Filter.eq('ADM0_NAME', 'India'))

**maskClouds** is a function to mask the satellite image, if the pixel has a "cloud_fraction" less than 0.5. This function is taken from [Ujaval Gandhi's GEE code.](https://code.earthengine.google.co.in/0f1259deeb86530cee552817a05e2031)

In [6]:
def maskClouds(image):
    mask = image.select('cloud_fraction').lt(0.1)
    return image.updateMask(mask)

# clip_image function clips the satellite image to our given area of interest (Chennai airshed box in our case)
# https://gis.stackexchange.com/questions/302760/gee-imagecollection-map-with-multiple-input-function
def clip_image(roi):
    def call_image(image):
        return image.clip(roi)
    return call_image

The satellite images are acquired from [**COPERNICUS/S5P/OFFL/L3_NO2**](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_NRTI_L3_NO2) service. NO2 data is provided in various terms (tropospheric_column_density, stratospheric_column_density etc). I'm taking tropospheric_column_density for now.

In [23]:
def download_tifs(pollutant):

    year=2019
    airshed_box, aoi = get_aoi('gridextents_shponly\\grids_delhi.shp')
    
    airshed_name = 'delhi'#airshed_shp.split('_')[2].split('.')[0]
    #tic = time.perf_counter()
    
    if year ==2022:
        max_month=8
    else:
        max_month=13
    for month in range(1,max_month):
        print(month)
    
        #Image Collection - l3_NO2 satellite -- SELECTING only two bands (NO2 Column Number density and Cloud_fraction)
        if pollutant=='SO2':
            band_name = 'SO2_column_number_density'
        elif pollutant == 'HCHO':
            band_name = 'tropospheric_HCHO_column_number_density'
        else:
            band_name = 'O3_column_number_density'
            
        collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_'+pollutant).select([band_name, 'cloud_fraction'])
        if month <9:
            startDate = str(year)+'-0'+str(month)+'-01'
            endDate = str(year)+'-0'+str(month)+'-15'
        elif month==9:
            startDate = str(year)+'-0'+str(month)+'-01'
            endDate = str(year)+'-'+str(month)+'-15'
        elif month<12:
            startDate = str(year)+'-'+str(month)+'-01'
            endDate = str(year)+'-'+str(month)+'-15'
        else:
            startDate = str(year)+'-'+str(month)+'-01'
            endDate = str(year)+'-'+str(month)+'-15'

        #if 'bishek/NO2_tifs\\'+airshed_name+'_15dayavg_'+'no2_'+startDate+'.tropospheric_NO2_column_number_density'+'.tif' in tifs:
         #   print('Yaoooo')
          #  continue
        #else:
         #   pass
        
        
        #Filter image collection -- filtered for date range, chennai_box range,
        fortnight=0
        while fortnight<2:
            filtered = collection.filter(ee.Filter.date(startDate, endDate)).filter(ee.Filter.bounds(aoi))
            #Apply the maskClouds and clip_image function to each image in the image collection.
            cloudMasked = filtered.map(maskClouds).select(band_name)
            clipped_images = cloudMasked.map(clip_image(aoi))
        
            #fortnightly mean
            image = clipped_images.mean()
        
            #Export image
            #geemap.ee_export_image(image, filename='bishek/NO2_tifs/'+airshed_name+'_15dayavg_'+'no2_'+startDate+'.tif',
             #                  scale=30,
              #                 region=aoi, file_per_band=True)
        
            ## To download aggregated data for the given airshed box in the form of a csv. Use 'toBands' of above to use this.
            geemap.zonal_statistics(clipped_images.toBands(), airshed_box,
                            'delhi/'+pollutant+'_csvs/'+airshed_name+'_15dayavg'+'_'+pollutant.lower()+'_'+startDate+'.csv', statistics_type='MEAN', scale=30)
        

            print(startDate+'xx'+endDate)
            startDate = startDate[:-2]+'16'
            if month==2:
                endDate = endDate[:-2]+'28'
            elif month in [4,6,9,11]:
                endDate = endDate[:-2]+'30'
            else:
                endDate = endDate[:-2]+'31'
        
            fortnight = fortnight+1

    
        # To download all tif images of a collection 
        #geemap.ee_export_image_collection(clipped_images, out_dir='tifs',
                                     #scale=30
         #                                )

    #toc = time.perf_counter()
    #print('Time taken {} seconds'.format(toc-tic))

In [24]:
from joblib import Parallel, delayed
import multiprocessing as mp
from multiprocessing.pool import ThreadPool

In [25]:
airsheds[19:20]

[]

In [26]:
pool= ThreadPool(processes=3)
pool.map(download_tifs,['SO2','HCHO','O3'])
#Parallel(n_jobs=3)(delayed(download_tifs)(year) for year in [2020,2021,2022])

1
Computing statistics ...
1
1
Computing statistics ...
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\HCHO_csvs\delhi_15dayavg_hcho_2019-01-01.csv
2019-01-01xx2019-01-15
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\HCHO_csvs\delhi_15dayavg_hcho_2019-01-16.csv
2019-01-16xx2019-01-31
2
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\HCHO_csvs\delhi_15dayavg_hcho_2019-02-01.csv
2019-02-01xx2019-02-15
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\HCHO_csvs\delhi_15dayavg_hcho_2019-02-16.csv
2019-02-16xx2019-02-28
3
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\HCHO_csvs\delhi_15dayavg_hcho_2019-03-01.csv
2019-03-01xx2019-03-15
Computing stati

Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\HCHO_csvs\delhi_15dayavg_hcho_2019-06-01.csv
2019-06-01xx2019-06-15
Computing statistics ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\O3_csvs\delhi_15dayavg_o3_2019-04-01.csv
2019-04-01xx2019-04-15
Computing statistics ...
Generating URL ...
Please wait ...
Generating URL ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\SO2_csvs\delhi_15dayavg_so2_2019-04-01.csv
2019-04-01xx2019-04-15
Computing statistics ...
Generating URL ...
Please wait ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\O3_csvs\delhi_15dayavg_o3_2019-04-16.csv
2019-04-16xx2019-04-30
5
Computing statistics ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\HCHO_csvs\delhi_15dayavg_hcho_2019-06-16.csv
2019-06-16xx2019-06-30
7
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\SO2_csvs\delhi_15dayavg_so2_2019-04-16.csv


Data downloaded to D:\Projects\UrbanEmissions Info\delhi\SO2_csvs\delhi_15dayavg_so2_2019-07-16.csv
2019-07-16xx2019-07-31
8
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\SO2_csvs\delhi_15dayavg_so2_2019-08-01.csv
2019-08-01xx2019-08-15
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\O3_csvs\delhi_15dayavg_o3_2019-08-01.csv
2019-08-01xx2019-08-15
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\HCHO_csvs\delhi_15dayavg_hcho_2019-10-01.csv
2019-10-01xx2019-10-15
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\SO2_csvs\delhi_15dayavg_so2_2019-08-16.csv
2019-08-16xx2019-08-31
9
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\O3_csvs\delhi_15dayavg_o

Data downloaded to D:\Projects\UrbanEmissions Info\delhi\SO2_csvs\delhi_15dayavg_so2_2019-11-16.csv
2019-11-16xx2019-11-30
12
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\O3_csvs\delhi_15dayavg_o3_2019-12-16.csv
2019-12-16xx2019-12-31
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\SO2_csvs\delhi_15dayavg_so2_2019-12-01.csv
2019-12-01xx2019-12-15
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to D:\Projects\UrbanEmissions Info\delhi\SO2_csvs\delhi_15dayavg_so2_2019-12-16.csv
2019-12-16xx2019-12-31


[None, None, None]

In [ ]:
tic = time.perf_counter()

#pool= ThreadPool(processes=10)
#pool.map(download_tifs_big,temp_airsheds)
download_tifs_big('gridextents_shponly\\grids_delhi.shp')
#Parallel(n_jobs=mp.cpu_count())(delayed(download_tifs)(airshed_shp,year) for airshed_shp in airsheds)

toc = time.perf_counter()
print('Time taken {} seconds'.format(toc-tic))

Ujaval Gandhi used "image.projection().nominalScale().getInfo()" as scale while exporting images. I'm not getting satisfactory results with that scale (huge pixels). So I used a scale of 30 and it gave me intuitively good results. Need help with tuning this parameter.

In [ ]:

## Following functions are useful when we download zonal statistics from GEE. 
def prep_dataset(csv_file,metric_name,separator,date_pos=0):
    df = pd.read_csv(csv_file)
    df = df.T.reset_index()[:-13]
    df = df.drop_duplicates()
    df['date']=df['index'].str.split(separator).str[date_pos]
    if df['date'][0][:1]=='A':
        df['year'] = df['date'].str[1:5]
        df['day'] = df['date'].str[5:]
        # converting to date
        df['date'] = pd.to_datetime(df['year'].astype(int) * 1000 + df['day'].astype(int), format='%Y%j')
        df = df.drop(['year','day'],axis=1)
    elif len(df['date'][4])>7:
        df['date']=df['date'].str[:4]+"-"+df['date'].str[4:6]+"-"+df['date'].str[6:]
    else:
         df['date']=df['date'].str[:4]+"-"+df['date'].str[4:6]+"-"+"01"
    
    df = df.drop(['index'],axis=1)
    
    df['date'] = pd.to_datetime(df['date'])
    df.columns = [metric_name,'date']
    
    df = df.groupby('date')[metric_name].mean().reset_index()
    df = df.set_index('date')
    
    df = df.fillna(np.NaN)
    return df

In [ ]:
## Preparing dataset
#global chennai_no2_df
#chennai_no2_df = prep_dataset('Chennai_NO2_2020-01-01_2020-01-31.csv','tropospheric_NO2_column_number_density','_')
#df = chennai_no2_df.resample('SMS').mean()
#df.index += pd.Timedelta(14, 'd')

In [ ]:
import glob
result = glob.glob("CLOUD*.csv")

In [ ]:
master = pd.DataFrame(columns=['date','tropospheric_NO2_column_number_density'])
for csv in result:
    df = prep_dataset(csv,'tropospheric_NO2_column_number_density','_')
    df = df.resample('SMS').mean()
    #df.index += pd.Timedelta(14, 'd')
    df = df.reset_index()
    master = master.append(df)


master = master.reset_index(drop=True).sort_values(by='date')
master.columns=['start_date','tropospheric_NO2_column_number_density']

In [ ]:
master.dropna().to_csv('CLOUDCHECK_Chennai_2021_NO2.csv',index=False)

In [ ]:
master.dropna()

In [ ]:
result

In [ ]:
img.close()

In [ ]:
import geemap

In [ ]:
print(geemap.__version__)

In [ ]:
import glob
tifs = glob.glob("NO2_tifs/*2020*.tif")

In [ ]:
cities = []
for tif in tifs:
    cities.append(tif.split('15')[0].split('\\')[1])

In [ ]:
df = pd.DataFrame(cities,columns=['city'])

In [ ]:
df.value_counts().to_csv('value_counts.csv')

In [ ]:
# 2022
# Entire Mumbai, Delhi and Indore missing.

In [ ]:
airshed_shp_temp = 'gridextents_shponly\\grids_patna.shp'
airshed_box, aoi = get_aoi(airshed_shp_temp)
airshed_name = airshed_shp_temp.split('_')[2].split('.')[0]
collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_HCHO').select(['tropospheric_HCHO_column_number_density', 'cloud_fraction'])

startDate = '2021-03-16'
endDate = '2021-03-31'

filtered = collection.filter(ee.Filter.date(startDate, endDate)).filter(ee.Filter.bounds(aoi))
#Apply the maskClouds and clip_image function to each image in the image collection.
cloudMasked = filtered.map(maskClouds).select('tropospheric_HCHO_column_number_density')
clipped_images = cloudMasked.map(clip_image(aoi))
#fortnightly mean
image = clipped_images.mean()
#Export image
geemap.ee_export_image(image, filename='HCHO_tifs/'+airshed_name+'_15dayavg_'+'hcho_'+startDate+'.tif',
                               scale=30,
                               region=aoi, file_per_band=True)
## To download aggregated data for the given airshed box in the form of a csv. Use 'toBands' of above to use this.
geemap.zonal_statistics(clipped_images.toBands(), airshed_box,
                        'HCHO_csvs/'+airshed_name+'_15dayavg'+'_hcho_'+startDate+'.csv', statistics_type='MEAN', scale=30)
        

In [ ]:
# startDate = '2021-01-01'
# endDate = '2021-01-15'

# startDate = '2021-01-16'
# endDate = '2021-01-31'

# startDate = '2021-02-01'
# endDate = '2021-02-15'

# startDate = '2021-02-16'
# endDate = '2021-02-28'

# startDate = '2021-03-01'
# endDate = '2021-03-15'

# startDate = '2021-03-16'
# endDate = '2021-03-31'

# startDate = '2021-04-01'
# endDate = '2021-04-15'

# startDate = '2021-04-16'
# endDate = '2021-04-30'

# startDate = '2021-05-01'
# endDate = '2021-05-15'

# startDate = '2021-05-16'
# endDate = '2021-05-31'


# startDate = '2021-06-06'
# endDate = '2021-06-30'

# startDate = '2021-07-01'
# endDate = '2021-07-15'



# startDate = '2021-08-16'
# endDate = '2021-08-31'


# startDate = '2021-09-01'
# endDate = '2021-09-15'


# startDate = '2021-09-16'
# endDate = '2021-09-30'

# startDate = '2021-10-01'
# endDate = '2021-10-15'

# startDate = '2021-10-16'
# endDate = '2021-10-31'

# startDate = '2021-11-01'
# endDate = '2021-11-15'

# startDate = '2021-11-16'
# endDate = '2021-11-30'

# startDate = '2021-12-01'
# endDate = '2021-12-15'

# startDate = '2021-12-16'
# endDate = '2021-12-31'

## Execute on Earth Engine directly

In [ ]:
def download_tifs_big(airshed_shp):

    year=2020
    airshed_box, aoi = get_aoi(airshed_shp)
    
    airshed_name = airshed_shp.split('_')[2].split('.')[0]
    #tic = time.perf_counter()
    
    if year ==2022:
        max_month=7
    else:
        max_month=13
    for month in range(1,max_month):
        print(month)
    
        #Image Collection - l3_NO2 satellite -- SELECTING only two bands (NO2 Column Number density and Cloud_fraction)
        collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_SO2').select(['SO2_column_number_density', 'cloud_fraction'])
        if month <9:
            startDate = str(year)+'-0'+str(month)+'-01'
            endDate = str(year)+'-0'+str(month)+'-15'
        elif month==9:
            startDate = str(year)+'-0'+str(month)+'-01'
            endDate = str(year)+'-'+str(month)+'-15'
        elif month<12:
            startDate = str(year)+'-'+str(month)+'-01'
            endDate = str(year)+'-'+str(month)+'-15'
        else:
            startDate = str(year)+'-'+str(month)+'-01'
            endDate = str(year)+'-'+str(month)+'-15'

        #Filter image collection -- filtered for date range, chennai_box range,
        fortnight=0
        while fortnight<2:
            filtered = collection.filter(ee.Filter.date(startDate, endDate)).filter(ee.Filter.bounds(aoi))
            #Apply the maskClouds and clip_image function to each image in the image collection.
            cloudMasked = filtered.map(maskClouds).select('SO2_column_number_density')
            clipped_images = cloudMasked.map(clip_image(aoi))
        
            #fortnightly mean
            image = clipped_images.mean()
        
            #Export image
            task = ee.batch.Export.image.toDrive(**{'image': image,
                                                    'description': 'Image Export {}'.format(airshed_name+'-'+str(fortnight)+'-'+str(month)+'-'+str(year)),
                                                    'fileNamePrefix': airshed_name+'_15dayavg_'+'so2_'+startDate+'.SO2_column_number_density',
                                                    'folder':'earthengine/SO2',
                                                    'scale': 30,
                                                    'region': aoi,
                                                    'maxPixels': 1e10
                                                   })
            task.start()
            
        
           
        

            print(startDate+'xx'+endDate)
            startDate = startDate[:-2]+'16'
            if month==2:
                endDate = endDate[:-2]+'29'
            elif month in [4,6,9,11]:
                endDate = endDate[:-2]+'30'
            else:
                endDate = endDate[:-2]+'31'
        
            fortnight = fortnight+1

    
        # To download all tif images of a collection 
        #geemap.ee_export_image_collection(clipped_images, out_dir='tifs',
                                     #scale=30
         #                                )

    #toc = time.perf_counter()
    #print('Time taken {} seconds'.format(toc-tic))

In [ ]:
tasks = ee.batch.Task.list()
for task in tasks:
    task_id = task.status()['id']
    task_state = task.status()['state']
    print(task_id, task_state)

In [ ]:
download_tifs_big('gridextents_shponly\\grids_delhi.shp')

In [ ]:
download_tifs_big('gridextents_shponly\\grids_indore.shp')

In [ ]:
download_tifs_big('gridextents_shponly\\grids_mumbai.shp')